# Google Art & Culture - Case study using CRISPS-DM

#### Autors: Manuel Alejandro Aponte, Cristian Beltran, Maria Paula Peña

In this notebook you will explore data and create classification models

## Objectives
The objective of this notebooks is:

* Visualize the data through graphs.
* Probar modelos de clasificación con el conjunto de datos.
* Optimize model by means of hyperparameters.

## Prerequisites

* Familiarity with Python, Sklearn & Grid Search. 
* Install python packages.

## Background 
This notebook belongs to Google Art & Culture Case Study using CRIPS-DM, where would be include process such as webscraping, exploratory data analysis, ML classificators and dashboards.

# TODO: 

**EDA**

**NN for classification**

**Stratify cross validation**

**Metric** 

# Exploration of data and Models 📊

## Libreries 📚

In [3]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity ="all"

In [19]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

## Load data

In [20]:
df = pd.read_csv('../data/final/pictures.csv')
df.head(5)

,index,category,filename,Title,Date Created,Dimensions,Artist,mean_grayscale,mean_blue,mean_green,mean_red,std_grayscale,std_blue,std_green,std_red
0,0,WHITE,0.jpg,"Symphony in White, No. 1 The White Girl",1862,overall 213 x 107.9 cm (83 7/8 x 42 1/2 in.) ...,James McNeill Whistler,0.707468,0.592548,0.702255,0.759860,0.113919,0.112658,0.117957,0.108062
1,1,WHITE,1.jpg,The Cradle,1872,w460 x h560 cm,Berthe Morisot,0.591144,0.519539,0.586287,0.626952,0.204013,0.186520,0.203947,0.212146
2,2,WHITE,2.jpg,The Magpie,1868 - 1869,w1300 x h890 mm,Claude Monet,0.687141,0.638097,0.687555,0.704309,0.133684,0.122419,0.133433,0.139909
3,3,WHITE,3.jpg,Summer evening on Skagen Sønderstrand,1893,100 × 150 cm,Peder Severin Krøyer,0.696685,0.713158,0.700890,0.682375,0.119145,0.071436,0.113651,0.153838
4,4,WHITE,4.jpg,"Composition with red, yellow and blue",1927,40 x 52 cm,Piet Mondriaan,0.836621,0.802547,0.829830,0.862469,0.246921,0.279878,0.261812,0.241762


In [21]:
y = df['category']
X = df[['mean_grayscale','mean_blue','mean_green','mean_red','std_grayscale','std_blue','std_green','std_red']] 

In [22]:
le = LabelEncoder()
y= le.fit_transform(y)
y


array([9, 9, 9, ..., 0, 0, 0])

## Models 📈

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def modelPipeline(X, y):
    
    default_params = {'random_state':42}
    
    log_reg = LogisticRegression(**default_params|{'max_iter':10000})
    knn = KNeighborsClassifier()
    svm = SVC(**default_params)
        
    models = [
            ('Logistic Regression', log_reg, {"penalty":['l2','none'], 'C':np.logspace(-4, 2, num=3)}), 
            ('K-Nearest Neighbors', knn, {'n_neighbors':[3,9,12], 'metric': ['l1','l2']}), 
            ('SVM', svm, {'C': [0.1, 1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel':['rbf','linear']}),
            ]
    
    results = pd.DataFrame(columns=['model','params','mean_test_score'])
    for name,model,params in models:
        classifier = GridSearchCV(model,params,cv=3,return_train_score=False)
        classifier.fit(X,y)
        full_report = pd.DataFrame(classifier.cv_results_)
        full_report['model']= name
        report = full_report[['model','params','mean_test_score','mean_fit_time']]
        results = pd.concat([results, report])
        
    return results

models_df = modelPipeline(X,y)
    

# Model Selection

### Search the models with best test scores

In [24]:
models_df.sort_values('mean_test_score',ascending=False)

,model,params,mean_test_score,mean_fit_time
40,SVM,"{'C': 1000, 'gamma': 1, 'kernel': 'rbf'}",0.833674,0.621838
30,SVM,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}",0.823467,0.408758
42,SVM,"{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}",0.820939,0.436639
49,SVM,"{'C': 1000, 'gamma': 0.0001, 'kernel': 'linear'}",0.817926,0.655841
47,SVM,"{'C': 1000, 'gamma': 0.001, 'kernel': 'linear'}",0.817926,0.654716
...,...,...,...,...
6,SVM,"{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}",0.265189,1.869613
8,SVM,"{'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}",0.265189,1.867203
28,SVM,"{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}",0.265189,1.867760
16,SVM,"{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}",0.265189,1.861052


In [25]:
models_df.groupby('model', as_index=False).max()

,model,mean_test_score,mean_fit_time
0,K-Nearest Neighbors,0.726256,0.011001
1,Logistic Regression,0.812871,5.213253
2,SVM,0.833674,1.871975


#### SVC (Super Vector Machine Classifier) 

In [15]:
X_train,X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, stratify=y,random_state=32)
# And here we train our model. IMPORTANT: we use kernel='linear'.
svc_model = SVC(kernel='linear', random_state=32)
svc_model.fit(X_train, y_train)

SVC(kernel='linear', random_state=32)

In [21]:
y_pred = svc_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[313,   4,  39,   6,   0,   0,   0,   0,   2,   0,   1],
       [ 16, 263,   0,   1,   0,   0,   0,   4,  59,  22,   0],
       [ 69,   0, 252,   9,   8,   0,   0,   4,   1,   0,  22],
       [ 16,   3,  17, 249,   1,   0,   0,   0,  44,  15,  20],
       [  1,   0,  17,   2, 136,   0,   0,  40,   0,   0,  47],
       [  0,   1,   0,   0,   0,   0,   0,  16,   0,   7,   2],
       [  1,  23,   0,   0,   0,   0,   0,  12,   0,   0,   0],
       [ 19,   8,  36,   2,  13,   0,   0, 221,   3,  11,   4],
       [  6,  56,   3,  30,   0,   0,   0,   1, 244,  23,   1],
       [  2,   4,   1,   3,   0,   0,   0,   3,   1, 307,  16],
       [  1,   1,  12,  26,  10,   0,   0,   1,   0,  31, 222]],
      dtype=int64)

In [20]:
# print prediction results
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       BLACK       0.70      0.86      0.77       365
        BLUE       0.72      0.72      0.72       365
       BROWN       0.67      0.69      0.68       365
       GREEN       0.76      0.68      0.72       365
      ORANGE       0.81      0.56      0.66       243
        PINK       0.00      0.00      0.00        26
      PURPLE       0.00      0.00      0.00        36
         RED       0.73      0.70      0.71       317
        TEAL       0.69      0.67      0.68       364
       WHITE       0.74      0.91      0.82       337
      YELLOW       0.66      0.73      0.69       304

    accuracy                           0.71      3087
   macro avg       0.59      0.59      0.59      3087
weighted avg       0.70      0.71      0.71      3087



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.602 total time=   2.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.581 total time=   2.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.585 total time=   2.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.601 total time=   2.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.597 total time=   2.1s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.399 total time=   3.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.384 total time=   2.9s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.390 total time=   3.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.403 total time=   3.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.380 total time=   3.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.140 total time=   3.3s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [27]:
grid.best_estimator_

SVC(C=1000, gamma=1)

In [25]:
grid_predictions = grid.predict(X_test)
 
# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

       BLACK       0.87      0.85      0.86       365
        BLUE       0.81      0.83      0.82       365
       BROWN       0.83      0.82      0.83       365
       GREEN       0.82      0.82      0.82       365
      ORANGE       0.84      0.84      0.84       243
        PINK       0.67      0.62      0.64        26
      PURPLE       0.78      0.69      0.74        36
         RED       0.84      0.84      0.84       317
        TEAL       0.77      0.79      0.78       364
       WHITE       0.91      0.92      0.91       337
      YELLOW       0.79      0.76      0.78       304

    accuracy                           0.83      3087
   macro avg       0.81      0.80      0.80      3087
weighted avg       0.83      0.83      0.83      3087



In [30]:
# https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html
#https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
# https://en.wikipedia.org/wiki/Radial_basis_function_kernel